# HW3

Submit via Slack. Due on Tuesday, April 13th, 2020, 6:29pm PST. You may work with one other person.

## TF-IDF

You are an analyst working at McDonalds as a store operations analyst, and charged with identifying areas for improvement for each franchise. Several metropolitan locations have been suffering recently from lower reviews.

Using the **mcdonalds-yelp-negative-reviews.csv** dataset, clean and parse the text reviews. Explain the decisions you make:
- why remove/keep stopwords?
- which stopwords to remove?
- stemming versus lemmatization?
- regex cleaning and substitution?
- adding in custom stopwords?
- what `n` for your `n-grams`?
- which words to collocate together?

Finally, generate a TF-IDF report that either **visualizes** or explains for a business (non-technical) stakeholder:
* the features your analysis showed that customers cited as reasons for a poor review
* the most common issues identified from your analysis that generated customer dissatisfaction.

Explain to what degree the TF-IDF findings make sense - what are its limitations?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
# import nltk
# nltk.download('wordnet')

In [ ]:
mac_df = pd.read_csv("mcdonalds-yelp-negative-reviews.csv", encoding="latin1")

In [ ]:
mac_df.head()

In [ ]:
# Make all lowercase
mac_df["review"] = mac_df["review"].str.lower()

In [ ]:
vectorizer = CountVectorizer(stop_words="english", binary=False)

X = vectorizer.fit_transform(mac_df["review"])

In [ ]:
vec_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names()).T

vec_df["num_count"] = vec_df.sum(axis=1)

In [ ]:
vec_df.sort_values("num_count", ascending=False)\
    .head(20)
#     .drop(list(set(stopwords.words('english'))), errors="ignore")

From the CountVectorizer, we can notice that many reviews contains "food", "order", "service", and "time"

## Regex Cleaning

In [ ]:
# Hamburger Variation
mac_df["review"] = mac_df["review"].str.replace(r"\w*\s*burgers?", "burger")

In [ ]:
# Big Macs
mac_df["review"] = mac_df["review"].str.replace(r"big\s*macs?", "burger")

Change types of burgers into burger to see how burgers served are reviewed

In [ ]:
# McDonald's
mac_df["review"] = mac_df["review"].str.replace(r"(?:\bmcdonald(?:'?s?)?\b)|(?:\bmcds?\b)", "mcdonald")

Changed variations of McDonald's to mcdonald to add to stopwords later

In [ ]:
# Punctuation Removal
mac_df["review"] = mac_df["review"].str.replace(r"[!|@|#|$|%|^|&|*|(|)|+|<|>|?|:|.|,|;|\"|\'|\\]", ' ')

In [ ]:
# Whitespace
mac_df["review"] = mac_df["review"].str.replace(r"\s{2,}", ' ')

In [ ]:
# Numbers
mac_df["review"] = mac_df["review"].str.replace(r"\d+\S*\d*\w*", "NUM_TOKEN")

Miscellaneous Regex

## Stemming

In [ ]:
# stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")

In [ ]:
def stmmer_func(review):
    tokens = [stemmer.stem(token) for token in review.split()]
    return ' '.join(tokens)

Reason for choosing SnowballStemmer: Stemming is more appropriate than Lemmetization in our analysis because we want to search for keywords that causes the bad reviews. For example, we want to know if the order was the issue for the customer. And words "ordered" and "order" are likely to indicate that the review has something to do with order. Also, I chose SnowballStemmer over PorterStemmer because it is the improved version. 

## Customize Stopwords

In [ ]:
stop_words = stopwords.words('english')

# Add stopword to stopwords list
stop_words.append("mcdonald")
stop_words = stop_words + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-", "!", "*", "?"]

# Remove from stopwords list
# stop_words.remove(["because", "most", "only"])

for w in ["because", "most", "only"]:
    stop_words.remove(w)

In [ ]:
"because" in stop_words

I removed "mcdonald" as we already know the review is about mcdonald. \
I added because the word(s) after "because" signals what causes customers to leave a bad review. Same logic applies to "most" and "only". 

## Collocation

In [ ]:
from nltk.corpus import wordnet

# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [ ]:
lemmatizer = WordNetLemmatizer()

mac_df["tokenized_words"] = mac_df["review"].apply(lemmatize_sentence)

In [ ]:
# Remove words in stopwords
mac_df["tokenized_words"] = mac_df["tokenized_words"].apply(lambda x: [word for word in x if word not in stop_words])

In [ ]:
collocation_finder = BigramCollocationFinder.from_documents(mac_df["tokenized_words"])
measures = BigramAssocMeasures()

collocation_finder.nbest(measures.raw_freq, 15)

These are the words that tend to collocate together. 

## TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(input="mcdonalds-yelp-negative-reviews.csv",
                         encoding="latin1",
                         lowercase=True,
                         stop_words=stop_words,
                         ngram_range=(3,3))

In [ ]:
corpus = list(mac_df["review"].values)

In [ ]:
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

In [ ]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [ ]:
score.head(15)

n-gram justification: I tried bigram for the analysis, but most words with high scores are usually collocated words like drive thru and ice cream. And quadgram simply has too many words that it does not appear in top rows. That is why I chose trigram. I acknowledge treating collocated words into one word will improve the analysis much better.

Most of the bad reviews are order related. Customers either did not get what they ordered or it took too long to get what they ordered, or even both. It seems like drive through and customer service need improvements as well.

It is consistent with the CountVectorizer analysis above as words like order, service were frequent words. However, none of tokens with high TF-IDF scores are food related as opposed to what we have observed in CounterVectorizer. There may be many reasons for that, but my assumption is that food is weighted down as they appear in many reviews. Also, there might be many variations around the word food, and thus not show up as much as the single word "food" does.  

## Product Attribution (Feature Engineering and Regex Practice)

Download the [dataset](https://dso-560-nlp-text-analytics.s3.amazonaws.com/truncated_catalog.csv) from the class S3 bucket (`dso560-nlp-text-analytics`).

In preparation for the group project, our client company has provided a dataset of women's clothing products they are considering cataloging. 

1. Filter for only **women's clothing items**.

2. For each clothing item:

* Identify its **category**:
```
Bottom
One Piece
Shoe
Handbag
Scarf
```
* Identify its **color**:
```
Beige
Black
Blue
Brown
Burgundy
Gold
Gray
Green
Multi 
Navy
Neutral
Orange
Pinks
Purple
Red
Silver
Teal
White
Yellow
```

Your output will be the same dataset, except with **3 additional fields**:
* `is_womens_clothing`
* `product_category`
* `colors`

`colors` should be a list of colors, since it is possible for a piece of clothing to have multiple colors.

In [ ]:
import re

In [ ]:
prod_df = pd.read_csv("truncated_catalog.csv")

In [ ]:
cols = prod_df.columns.to_list()

for col in cols:
    prod_df[col] = prod_df[col].str.lower()

In [ ]:
# Fill Null Values
prod_df.iloc[:, :7] = prod_df.iloc[:, :7].fillna('')

In [ ]:
prod_df.head(3)

## is_womens_clothing

In [ ]:
# Regex for capturing women related words 
woman_exp = "\bwi(?:fe|ves)|girls?|wom(?:a|e)n|lad(?:y|ies)|madams?|brides?|widows?|females?|femini\w*|maternal\w*|moms?\b"

# Search all columns
for col in cols:
    prod_df[f"is_womens_clothing_{col}"] = False
    
    # Find if women related words exist in the column
    prod_df[f"is_womens_clothing_{col}"] = prod_df[col].str.contains(woman_exp, case=False, flags=re.IGNORECASE, regex=True)
        
    print(f"{col} searched")
    
# If any of is_womens_clothing is True, then is_womens_clothing is True. Otherwise False
prod_df[f"is_womens_clothing"] = prod_df.iloc[:, 7:].any(axis=1)

# Drop other intermediate columns
col_to_drop = prod_df.iloc[0, 7:-1].index.to_list()
prod_df.drop(col_to_drop, axis=1, inplace=True)

## product_category

In [ ]:
# Expressions
bottom_exp = "(?:baggies|bottom|pant|jean|cord|chino|denim|legging|overall|short|trouser)(?:es|s)?"
one_piece_exp = "\bone[\S|\s]?piece|\w*dress|all[\S|\s]?in[\S|\s]?one\b"
shoe_exp = "(?:shoe|boot|cleat|hopper|trainer|flat|flip[\S|\s]?flop|heel|pump|slide|slipper|skate|sneaker|wedge)(?:s|es)?"
handbag_exp = "(?:\w* ?bags?|clutch(?:es)?|satchels?)"
scarf_exp = "(?:\w* ?scar(?:f|(?:ves))?|snoods?|stoles?|boas?|sarongs?)"

cats_list = ["Bottom", "One_Piece", "Shoe", "Handbag", "Scarf"]
exps_list = [bottom_exp, one_piece_exp, shoe_exp, handbag_exp, scarf_exp]

# For each product category
for cat, exp in zip(cats_list, exps_list):
    prod_df.loc[:,cat] = 0
    
    for col in cols:
        # Add the number of occurrences in all columns
        prod_df[cat] = prod_df[col].str.findall(exp, flags=re.IGNORECASE).apply(lambda x: len(x))
        print(f"{cat} in {col} searched")

In [ ]:
# Finds the category that has the highest score
prod_df["product_category"] = prod_df.iloc[:, 8:].apply(lambda x: x.idxmax() if x.sum() != 0 else None, axis=1)

Acknowledgement: idxmax fails to identify the category when there is a tie. idxmax fails to break the tie as it chooses the index of former tie. For example, if both shoe and handbag show up once in a product, it idxmax will choose shoe instead of handbag, which may not be true.

In [ ]:
prod_df.drop(["Bottom", "One_Piece", "Shoe", "Handbag", "Scarf"], axis=1, inplace=True)

In [ ]:
prod_df.head(3)

## colors

In [ ]:
color_exp = "(?:Beige|Black|Blue|Brown|Burgund(?:y|ies)|Gold|Gra(?:y|ies)|Green|Multi|Nav(?:y|ies)|Neutral|Orange|Pink|Purple|Red|Silver|Teal|White|Yellow)s?"

for col in cols:
    prod_df[f"colors_{col}"] = None
    
    # Find colors
    prod_df[f"colors_{col}"] = prod_df[col].str.findall(color_exp, flags=re.IGNORECASE).apply(lambda x: ''.join(x))
        
    print(f"{col} searched")

In [ ]:
# Append colors to the list
prod_df["colors"] = prod_df.iloc[:, 9:].apply(lambda x: list(set([color for color in x if color != ''])), axis=1)

In [ ]:
# Drop columns
prod_df.drop(["colors_brand", "colors_name", "colors_description", "colors_brand_category",
              "colors_brand_canonical_url", "colors_details", "colors_tsv"],
             axis=1, inplace=True)

In [ ]:
# Change color to "Multi" if multiple colors in a product
prod_df["colors"] = prod_df["colors"].apply(lambda x: x if len(x) < 2 else ["Multi"])
prod_df["colors"] = prod_df["colors"].apply(lambda x: None if len(x) == 0 else x)

In [ ]:
prod_df